In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize 
from gensim.utils import tokenize
from nltk.tokenize import sent_tokenize
import pandas as pd
import nltk
from scipy.spatial import distance

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import time
import codecs
import csv
from sklearn import feature_extraction
import collections 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.manifold import MDS

from scipy.spatial import distance
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import nltk.stem as stemmer

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def rem_stop_words(example_sent):
    
    example_sent = str(example_sent)
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(example_sent) 
  
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
  
    return(filtered_sentence) 

# Take in a string and remove punctuation, repeat lines, etc.
# RETURN: string (cleaned up)
def cleaner(text):
    
    text = re.sub('NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO PROTECT CONFIDENTIALITY','', text)
    text = re.sub('<BR/>',' ', text)
    text = text.lower()
    text = re.sub('[****]','',text)
    
    text = re.sub('[/%#;:!,.<>\'?\"()-\[\]]',' ',text)
    text = re.sub('[0-9*]','',text)

    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text)
    
    a = nltk.pos_tag(tokenize_only(text))
    a = pd.DataFrame(a)
    a = a[a[1] == 'NN']
    text = ' '.join(a[0])
    
    return(text)
def to_lower(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    re.sub(r"\S*https?:\S*", "", text)
    text = text.lower()
    
    return(text)

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if(stemmer.stem(token) not in rem_words):
                filtered_tokens.append(token)
    return filtered_tokens

def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    stems = [e for e in stems if e not in rem_words ]
    return stems
stemmer = SnowballStemmer("english")

In [4]:
rem_words_1 = "survey,nurse,cited,am,pm,resident,residents,she,not,review,facility,state,have,year,employee,reported,management,"
rem_words_2 = "he,staff,interview,care,date,reveal,record,be,month,document,physician,lpn,na,code,description,previous,hospital,m,"
rem_words_3 = "observation,stated,assessment,don,mds,gna,cna,pa,cm,medical,names,order,status,immediately,redacted,condition,plan,"
rem_words_4 = "administrative,note,time,indicated,incident,redirected,director,uses,include,inr,rn,investigation,asked,following,"
rem_words_5 = "provided,alleged,failed,identified,said,ei,ri,parking,assist,every,get,"
rem_words_6 = "room,left,mm,hg,adl,xl,day,mg,mar,knows,prn,look,ard,give,ml,also,ted,made,non,bims,pasrr," # 7/13 update
num_alphas  = "one,two,three,four,five,six,seven,eight,nine,ten,"
#rem_words = rem_words_1 + rem_words_2 + rem_words_3 + rem_words_4 + rem_words_5 + rem_words_6 + num_alphas
rem_words = num_alphas
rem_words = rem_words.split(',')
rem_words = [stemmer.stem(t) for t in rem_words]

In [5]:
feb20_dataset = pd.read_csv("February20_Twitter_Data.csv")
jul20_dataset = pd.read_csv("July20_Twitter_Data.csv")
sep21_dataset = pd.read_csv("September21_Twitter_Data.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
feb20_dataset.head(2)

,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink
0,Sat Feb 01 00:15:16 +0000 2020,1.223400e+18,1.223400e+18,This lady is on her 5th Corona,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 30]",NaN
1,Sat Feb 01 00:19:24 +0000 2020,1.223400e+18,1.223400e+18,"Empress of China is closed, pls don’t text","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 42]",NaN


In [7]:
jul20_dataset.head(2)

,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink
0,Wed Jul 01 01:11:02 +0000 2020,1.280000e+18,1.280000e+18,"@atrupar Evil @realDonaldTrump: ""I watched Ame...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,1.280000e+18,1.280000e+18,288277167.0,288277167.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[9, 286]",NaN
1,Wed Jul 01 03:58:30 +0000 2020,1.280000e+18,1.280000e+18,For the win... https://t.co/xsmAYHOxHt,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 14]","{'url': 'https://t.co/xsmAYHOxHt', 'expanded':..."


In [8]:
sep21_dataset.head(2)

,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink
0,Wed Sep 01 21:27:20 +0000 2021,1.433180e+18,1.433180e+18,@CrisLaderas What evidence does the province h...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.433170e+18,1.433170e+18,5.139697e+07,5.139697e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13, 179]",NaN
1,Wed Sep 01 21:38:46 +0000 2021,1.433180e+18,1.433180e+18,@physiocara @MrMcgill8 @j_mcelroy Since everyo...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,1.432810e+18,1.432810e+18,3.168082e+09,3.168082e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[34, 258]",NaN


In [9]:
feb20_tweets = feb20_dataset["full_text"]
jul20_tweets = jul20_dataset["full_text"]
sep21_tweets = sep21_dataset["full_text"]

In [11]:
print(feb20_tweets[4],jul20_tweets[4],sep21_tweets[4], sep = '\n\n')

In the race for the White House, the China question goes unanswered
https://t.co/kN9uFSXAUB https://t.co/aodHTUVKz8

@marky_b_tweets Always. I really want us to succeed, but I also recognize the terrifying reality of this virus.
It's very very good at spreading in human hosts. I love it for that while also wanting it to go away.

Oakland County public health officials are aiming to vaccinate school-age children and college students ahead of schools opening up. https://t.co/33QAy3uJFY


In [12]:
temp_feb20 = feb20_tweets.str.replace('\n',' ')
temp_jul20 = jul20_tweets.str.replace('\n',' ')
temp_sep21 = sep21_tweets.str.replace('\n',' ')
for k in range(len(temp_feb20)):
    temp_feb20[k] = re.sub(r"\S*https?:\S*","", temp_feb20[k])
for k in range(len(temp_jul20)):
    temp_jul20[k] = re.sub(r"\S*https?:\S*","", temp_jul20[k])
for k in range(len(temp_sep21)):
    temp_sep21[k] = re.sub(r"\S*https?:\S*","", temp_sep21[k])

In [13]:
cleaned_feb20_tweets = []
cleaned_jul20_tweets = []
cleaned_sep21_tweets = []

In [14]:
for k in temp_feb20: cleaned_feb20_tweets.append(''.join(to_lower(k)))
for k in temp_jul20: cleaned_jul20_tweets.append(''.join(to_lower(k)))
for k in temp_sep21: cleaned_sep21_tweets.append(''.join(to_lower(k)))

In [16]:
print(temp_feb20[1], cleaned_feb20_tweets[1], sep = '\n', end = '\n\n') 
print(temp_jul20[4], cleaned_jul20_tweets[4], sep = '\n', end = '\n\n') 
print(temp_sep21[4], cleaned_sep21_tweets[4], sep = '\n', end = '\n\n') 

Empress of China is closed, pls don’t text
empress of china is closed, pls don’t text

@marky_b_tweets Always. I really want us to succeed, but I also recognize the terrifying reality of this virus. It's very very good at spreading in human hosts. I love it for that while also wanting it to go away.
@marky_b_tweets always. i really want us to succeed, but i also recognize the terrifying reality of this virus. it's very very good at spreading in human hosts. i love it for that while also wanting it to go away.

Oakland County public health officials are aiming to vaccinate school-age children and college students ahead of schools opening up. 
oakland county public health officials are aiming to vaccinate school-age children and college students ahead of schools opening up. 



In [17]:
feb20_vectors = [nltk.word_tokenize(cleaned_feb20_tweets) for cleaned_feb20_tweets in cleaned_feb20_tweets]
jul20_vectors = [nltk.word_tokenize(cleaned_jul20_tweets) for cleaned_jul20_tweets in cleaned_jul20_tweets]
sep21_vectors = [nltk.word_tokenize(cleaned_sep21_tweets) for cleaned_sep21_tweets in cleaned_sep21_tweets]

In [18]:
feb20_model = Word2Vec(feb20_vectors)
jul20_model = Word2Vec(jul20_vectors)
sep21_model = Word2Vec(sep21_vectors)

In [32]:
feb20_model.wv.most_similar("corona", topn=20)

[('carona', 0.5483368635177612),
 ('deadly', 0.4830470085144043),
 ('ncov', 0.4709779918193817),
 ('corana', 0.46261659264564514),
 ('coronavirus', 0.45818108320236206),
 ('bacteria', 0.436644047498703),
 ('bleach', 0.4309340715408325),
 ('corono', 0.4233631491661072),
 ('plagued', 0.4198324978351593),
 ('🦠', 0.41473135352134705),
 ('spreading', 0.41175413131713867),
 ('karona', 0.40684470534324646),
 ('crona', 0.4053245186805725),
 ('itself', 0.39973771572113037),
 ('stomach', 0.3957355320453644),
 ('shedding', 0.3948439359664917),
 ('unknown', 0.3941563367843628),
 ('coronas', 0.3892984986305237),
 ('korona', 0.38918015360832214),
 ('cold', 0.3891006410121918)]

In [33]:
jul20_model.wv.most_similar('corona', topn=20)# cdc, 

[('covid', 0.6260733604431152),
 ('covid-19', 0.5102304220199585),
 ('rona', 0.4800625145435333),
 ('wuhan', 0.4728194773197174),
 ('china', 0.4653492569923401),
 ('covid19', 0.44503939151763916),
 ('..', 0.4445585310459137),
 ('cancer', 0.4335877597332001),
 ('🦠', 0.4292427897453308),
 ('chinese', 0.41630804538726807),
 ('flu', 0.40979623794555664),
 ('ccp', 0.3966471552848816),
 ('humans', 0.3937920928001404),
 ('covid-', 0.38671696186065674),
 ('heat', 0.3854938745498657),
 ('pigs', 0.3768843412399292),
 ('bacteria', 0.37649625539779663),
 ('pneumonia', 0.375264972448349),
 ('this', 0.36885419487953186),
 ('the', 0.3681204319000244)]

In [34]:
sep21_model.wv.most_similar('covid', topn=20)# cdc, 

[('covid-19', 0.7328019738197327),
 ('virus', 0.6041736602783203),
 ('cancer', 0.5669732689857483),
 ('flu', 0.5638599991798401),
 ('jab', 0.5367597341537476),
 ('disease', 0.5228464603424072),
 ('term', 0.4962363839149475),
 ('symptoms', 0.4938730001449585),
 ('antibodies', 0.4809015691280365),
 ('taken', 0.4798889458179474),
 ('low', 0.47623658180236816),
 ('hospital', 0.47212570905685425),
 ('richer', 0.47064316272735596),
 ('number', 0.4683408737182617),
 ('illness', 0.4670003056526184),
 ('poke', 0.46253252029418945),
 ('ivermectin', 0.45924487709999084),
 ('being', 0.45250195264816284),
 ('intubated', 0.45083945989608765),
 ('chance', 0.4503765404224396)]

In [ ]:
distance.cosine(feb_model.wv['corona'] , feb_model.wv['disease'])

In [ ]:
distance.cosine(feb_model.wv['corona'] , july_model.wv['corona'])